In [14]:
import numpy as np 
import pandas as pd 

In [4]:
data = pd.read_csv("pageviews.csv",
                   parse_dates=["FEC_EVENT"])
data

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID
0,2018-03-30 07:35:48,1,1,1,1,1,1,0
1,2018-03-30 07:35:52,2,2,2,2,2,1,0
2,2018-03-30 07:36:11,3,2,2,2,3,1,0
3,2018-03-30 07:36:16,4,2,2,2,3,1,0
4,2018-03-30 07:41:38,5,2,2,2,2,1,0
5,2018-03-30 07:41:42,2,2,2,2,2,1,0
6,2018-03-30 07:42:01,3,2,2,2,3,1,0
7,2018-03-30 07:42:05,4,2,2,2,3,1,0
8,2018-03-30 07:43:43,3,2,2,2,3,1,0
9,2018-03-30 07:44:14,6,2,2,2,3,1,0


In [5]:
X_test = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test = pd.concat(X_test, axis=1)

haciendo PAGE
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_TOP
haciendo CONTENT_CATEGORY_BOTTOM
haciendo SITE_ID
haciendo ON_SITE_SEARCH_TERM


In [6]:
data = data[data.FEC_EVENT.dt.month < 10]
X_train = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train = pd.concat(X_train, axis=1)

haciendo PAGE
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_TOP
haciendo CONTENT_CATEGORY_BOTTOM
haciendo SITE_ID
haciendo ON_SITE_SEARCH_TERM


In [7]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

In [10]:
y_prev = pd.read_csv("conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(
        set(X_train.index))
y_train.loc[list(idx)] = 1

In [15]:
from xgboost.sklearn import XGBClassifier

In [19]:
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score

fi = []
test_probs = []
i = 0
for train_idx, valid_idx in model_selection.KFold(n_splits=4, shuffle=True).split(X_train):
    i += 1
    Xt = X_train.iloc[train_idx]
    yt = y_train.loc[X_train.index].iloc[train_idx]

    Xv = X_train.iloc[valid_idx]
    yv = y_train.loc[X_train.index].iloc[valid_idx]

    learner = XGBClassifier(n_estimators=1000)
    learner.fit(Xt, yt,  early_stopping_rounds=7, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)])
    
    test_probs.append(pd.Series(learner.predict_proba(X_test)[:, -1],
                                index=X_test.index, name="fold_" + str(i)))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
test_probs.index.name="USER_ID"
test_probs.name="SCORE"
test_probs.to_csv("ramiro3.csv", header=True)
fi = pd.concat(fi, axis=1).mean(axis=1)

[0]	validation_0-auc:0.767032	validation_1-auc:0.775246
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 6 rounds.
[1]	validation_0-auc:0.774398	validation_1-auc:0.779585
[2]	validation_0-auc:0.78323	validation_1-auc:0.78238
[3]	validation_0-auc:0.784673	validation_1-auc:0.781861
[4]	validation_0-auc:0.819444	validation_1-auc:0.787583
[5]	validation_0-auc:0.823362	validation_1-auc:0.786465
[6]	validation_0-auc:0.827718	validation_1-auc:0.784722
[7]	validation_0-auc:0.834684	validation_1-auc:0.785347
[8]	validation_0-auc:0.836477	validation_1-auc:0.78576
[9]	validation_0-auc:0.837315	validation_1-auc:0.785826
[10]	validation_0-auc:0.845097	validation_1-auc:0.804099
[11]	validation_0-auc:0.846422	validation_1-auc:0.80334
[12]	validation_0-auc:0.846569	validation_1-auc:0.806029
[13]	validation_0-auc:0.850149	validation_1-auc:0.805563
[14]	validation_0-auc:0.855282	validation_1-auc:0.804804
[15

[35]	validation_0-auc:0.891686	validation_1-auc:0.829407
[36]	validation_0-auc:0.892583	validation_1-auc:0.827985
[37]	validation_0-auc:0.893226	validation_1-auc:0.827361
[38]	validation_0-auc:0.894264	validation_1-auc:0.828423
[39]	validation_0-auc:0.896845	validation_1-auc:0.827868
[40]	validation_0-auc:0.897512	validation_1-auc:0.82778
[41]	validation_0-auc:0.901134	validation_1-auc:0.825998
Stopping. Best iteration:
[35]	validation_0-auc:0.891686	validation_1-auc:0.829407

[0]	validation_0-auc:0.773663	validation_1-auc:0.704494
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 6 rounds.
[1]	validation_0-auc:0.780596	validation_1-auc:0.708672
[2]	validation_0-auc:0.787428	validation_1-auc:0.719555
[3]	validation_0-auc:0.807739	validation_1-auc:0.715504
[4]	validation_0-auc:0.838694	validation_1-auc:0.734177
[5]	validation_0-auc:0.838201	validation_1-auc:0.735391
[6]	validation_0-auc:0.842

[26]	validation_0-auc:0.881153	validation_1-auc:0.792801
[27]	validation_0-auc:0.883264	validation_1-auc:0.791767
[28]	validation_0-auc:0.884664	validation_1-auc:0.792549
[29]	validation_0-auc:0.888129	validation_1-auc:0.789638
[30]	validation_0-auc:0.888882	validation_1-auc:0.792524
[31]	validation_0-auc:0.889466	validation_1-auc:0.792612
Stopping. Best iteration:
[25]	validation_0-auc:0.880383	validation_1-auc:0.793519

[0]	validation_0-auc:0.777567	validation_1-auc:0.723977
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 6 rounds.
[1]	validation_0-auc:0.778067	validation_1-auc:0.723958
[2]	validation_0-auc:0.821202	validation_1-auc:0.767317
[3]	validation_0-auc:0.824836	validation_1-auc:0.76487
[4]	validation_0-auc:0.832472	validation_1-auc:0.767794
[5]	validation_0-auc:0.841153	validation_1-auc:0.761716
[6]	validation_0-auc:0.848515	validation_1-auc:0.763417
[7]	validation_0-auc:0.8496